## Global Terrorism Attribution Predictor

In this project I...

Imports and field variables

In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
data_path = "./data"

Download Global Terrorism Database from Kaggle

In [2]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files("START-UMD/gtd", quiet=False, path=data_path)

gtd.zip: Skipping, found more recently modified local copy (use --force to force download)


Read data into Pandas DataFrame

In [3]:
# skipcols=lambda x: x not in ["extended", "addnotes", "scite1", "scite2", "scite3", "dbsource"]
usecols=["iyear",
         "suicide",
         "imonth",
         "country",
         "region",
         "provstate",
         "city",
         "alternative",
         "attacktype1",
         "targtype1",
         "natlty1",
         "gname",
         "weaptype1",
         "ransom"]

data = pd.read_csv(data_path + "/gtd.zip",
                        encoding = "ISO-8859-1",
                        compression='zip',
                        usecols=usecols)

set dTypes

In [4]:
# data = pd.DataFrame(data.dropna())
# data = data.fillna(value={0})
data['alternative'] = data.alternative.astype('category')
data['suicide'] = data.suicide.astype('category')
data['iyear'] = data.iyear.astype('category')
data['imonth'] = data.imonth.astype('category')
data['country'] = data.country.astype('category')
data['region'] = data.region.astype('category')
data['provstate'] = data.provstate.astype('str').astype('category')
data['city'] = data.city.astype('str').astype('category')
data['attacktype1'] = data.attacktype1.astype('category')
data['targtype1'] = data.targtype1.astype('category')
data['natlty1'] = data.natlty1.astype('category')
data['weaptype1'] = data.weaptype1.astype('category')
data['ransom'] = data.ransom.astype('category')

In [5]:
pd.set_option('display.max_columns', None) 
data.head()

,iyear,imonth,country,region,provstate,city,alternative,suicide,attacktype1,targtype1,natlty1,gname,weaptype1,ransom
0,1970,7,58,2,nan,Santo Domingo,NaN,0,1,14,58.0,MANO-D,13,0.0
1,1970,0,130,1,Federal,Mexico city,NaN,0,6,7,21.0,23rd of September Communist League,13,1.0
2,1970,1,160,5,Tarlac,Unknown,NaN,0,1,10,217.0,Unknown,13,0.0
3,1970,1,78,8,Attica,Athens,NaN,0,3,7,217.0,Unknown,6,0.0
4,1970,1,101,4,Fukouka,Fukouka,NaN,0,7,7,217.0,Unknown,8,0.0


In [6]:
data.describe()

,iyear,imonth,country,region,provstate,city,alternative,suicide,attacktype1,targtype1,natlty1,gname,weaptype1,ransom
count,181691,181691,181691,181691,181691,181691,29011.0,181691,181691,181691,180132.0,181691,181691,77381.0
unique,47,13,205,12,2856,36675,5.0,2,9,22,215.0,3537,12,3.0
top,2014,5,95,10,Baghdad,Unknown,1.0,0,3,14,95.0,Unknown,6,0.0
freq,16903,16875,24636,50474,7645,9775,23410.0,175058,88255,43511,24113.0,82782,92426,74684.0


Filter out unattributed attacks

In [7]:
attributed = data.loc[data['gname'] != "Unknown"]

In [8]:
# attributed = pd.DataFrame(attributed.dropna())
attributed = attributed.iloc[:10000]
attributed = attributed.sample(frac=1)
attributed.head()

,iyear,imonth,country,region,provstate,city,alternative,suicide,attacktype1,targtype1,natlty1,gname,weaptype1,ransom
3971,1976,7,26,3,La Paz,La Paz,NaN,0,3,2,26.0,Revolutionary Workers Party of Bolivia (PRTB),6,0.0
5924,1978,2,603,8,Northern Ireland,Belfast,NaN,0,3,8,233.0,Irish Republican Army (IRA),6,0.0
8179,1979,4,185,8,Basque Country,Durango,NaN,0,1,3,185.0,Basque Fatherland and Freedom (ETA),5,0.0
12513,1980,12,78,8,Attica,Athens,NaN,0,3,1,78.0,October-80,6,0.0
6019,1978,2,160,5,Zamboanga del Sur,Zamboanga City,NaN,0,6,2,69.0,Moro National Liberation Front (MNLF),13,1.0


In [9]:
to_fit = attributed.select_dtypes(include=['object'])
le = preprocessing.LabelEncoder()
attributed = attributed.apply(le.fit_transform)
attributed.head()

,iyear,imonth,country,region,provstate,city,alternative,suicide,attacktype1,targtype1,natlty1,gname,weaptype1,ransom
3971,6,7,11,2,273,989,4005,0,2,1,12,711,4,0
5924,8,2,107,6,391,221,5559,0,2,7,106,392,4,0
8179,9,4,79,6,57,582,5560,0,0,2,84,144,3,0
12513,10,12,31,6,41,119,5561,0,2,0,34,542,4,0
6019,8,2,69,4,572,2026,5562,0,5,1,32,491,11,1


In [10]:
# 1. INSTANTIATE
# enc = preprocessing.OneHotEncoder()

# # 2. FIT
# enc.fit(attributed)

# 3. Transform
# onehotlabels = enc.transform(attributed).toarray()
# attributed = pd.get_dummies(attributed)
# attributed.shape

Making a copy of the attributeddataframe which does not contain the dependent variable

In [11]:
independent_vars = attributed.copy()
independent_vars = independent_vars[independent_vars.columns.drop(list(independent_vars.filter(regex='gname')))]
independent_vars.shape

(10000, 13)

Making a dataframe with of the dependent variables (organizations to which the attacks were attribued)

In [12]:
dependent_vars = pd.DataFrame(attributed.loc[:,list(attributed.filter(regex='gname'))])
dependent_vars.shape

(10000, 1)

Split the data into training and test sets

In [13]:
independant_size = independent_vars.shape
dependant_size = dependent_vars.shape
if independant_size[0] != dependant_size[0]:
    raise ValueError('Independent and Dependant DFs do not match')

split_size = int(independant_size[0] * 0.7)

training_independant_vars = independent_vars.iloc[:split_size]
test_independant_vars = independent_vars.iloc[split_size:]
print(training_independant_vars.shape)
print(test_independant_vars.shape)

training_dependant_vars = dependent_vars.iloc[:split_size]
test_dependant_vars = dependent_vars.iloc[split_size:]

print(training_dependant_vars.shape)
print(test_dependant_vars.shape)

(7000, 13)
(3000, 13)
(7000, 1)
(3000, 1)


TODO: Implement Training model

In [14]:
reg = RandomForestClassifier(verbose=10, n_estimators=15, warm_start=True)
reg.fit(training_independant_vars, training_dependant_vars.values.ravel())
# reg.fit(train_fold, train_y.values.ravel())
predictions = reg.predict(test_independant_vars)
total = accuracy_score(test_dependant_vars, predictions)
print("Accuracy is {}%".format(total * 100))


building tree 1 of 15
building tree 2 of 15
building tree 3 of 15
building tree 4 of 15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.4s remaining:    0.0s


building tree 5 of 15
building tree 6 of 15
building tree 7 of 15
building tree 8 of 15


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.5s remaining:    0.0s


building tree 9 of 15
building tree 10 of 15
building tree 11 of 15
building tree 12 of 15
building tree 13 of 15
building tree 14 of 15
building tree 15 of 15
Accuracy is 61.56666666666667%


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.2s finished


In [15]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, format(cm[i, j], fmt),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.tight_layout()

In [16]:
from sklearn.metrics import confusion_matrix
print("Predictions: ", predictions)

Predictions:  [392 582  37 ... 391 247 822]


In [17]:
# Compute confusion matrix
import numpy as np
cnf_matrix = confusion_matrix(test_dependant_vars, predictions)
np.set_printoptions(precision=2)

In [20]:
# # Plot non-normalized confusion matrix
import itertools
class_names = test_independant_vars.columns
# print(class_names)
plt.figure()
plot_confusion_matrix(cnf_matrix, class_names, title='Confusion matrix, without normalization')


Confusion matrix, without normalization
[[0 0 0 ... 0 0 0]
 [0 9 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 2 0 0]]


<Figure size 432x288 with 0 Axes>

In [19]:
plt.show()